In [ ]:
import sqlite3
import pandas as pd

def print_table(query, p=False):
    conn = sqlite3.connect('../data/ratingRager.db')
    cursor = conn.cursor()
    cursor.execute(query)

    column_names = [description[0] for description in cursor.description]

    rows = cursor.fetchall()

    column_widths = [max(len(str(row[i])) for row in rows + [column_names]) for i in range(len(column_names))]

    header = "  ".join(f"{name:<{column_widths[i]}}" for i, name in enumerate(column_names))
    if p:
        print('\n',header)
        print("-" * len(header))

        for row in rows:
            row_str = "  ".join(f"{str(item):<{column_widths[i]}}" for i, item in enumerate(row))
            print(row_str)
        print('\n')

    return pd.read_sql_query(query, conn)

In [ ]:
QUERY = """
WITH RecentReviews AS (
    SELECT
        r.restaurant_id,
        r.rating,
        r.date,
        ROW_NUMBER() OVER (
            PARTITION BY r.restaurant_id
            ORDER BY r.date DESC
        ) AS review_rank
    FROM reviews r
)
SELECT
    rr.restaurant_id,
    r.name,
    AVG(rr.rating) AS current_rating,
    r.rating AS average_rating
FROM RecentReviews rr
JOIN restaurants r on r.id = rr.restaurant_id
WHERE rr.review_rank <= 5
GROUP BY rr.restaurant_id
"""
df = print_table(QUERY)

In [ ]:
df['trend_score'] = (((df['current_rating'] - df['average_rating']) / df['average_rating'] ) + 1 ) / 2

In [ ]:
# Update score table

conn = sqlite3.connect('../data/ratingRager.db')
cursor = conn.cursor()

for index, row in df.iterrows():
    cursor.execute(
        "UPDATE scores SET trend_score = ? WHERE restaurant_id = ?;",
        (row['trend_score'], row['restaurant_id'])
    )

conn.commit()
conn.close()